**RC6** — симметричный блочный криптографический алгоритм, производный от алгоритма RC5. Был создан Роном Ривестом, Мэттом Робшау и Рэем Сиднеем для удовлетворения требований конкурса Advanced Encryption Standard (AES). Алгоритм был одним из пяти финалистов конкурса, был также представлен NESSIE и CRYPTREC.

Вариант шифра RC6, заявленный на конкурс AES, поддерживает блоки длиной 128 бит и ключи длиной 128, 192 и 256 бит, но сам алгоритм, как и RC5, может быть сконфигурирован для поддержки более широкого диапазона длин как блоков, так и ключей (от 0 до 2040 бит). 

[Wiki](https://ru.wikipedia.org/wiki/RC6)

In [27]:
import math

In [28]:
# Округление до ближайшего нечетного целого:
def odd(x):
  if math.trunc(x) % 2 == 0:
    y = math.ceil(x)
  else:
    y = math.floor(x)
  return y


In [29]:
# Циклический сдвиг вправо:
def right_shift(x, n):
  n = n & (2 ** math.floor(math.log2(w)) - 1)
  return ((x >> n) | (x << (w - n))) & (2 ** w - 1)

# Циклический сдвиг влево:
def left_shift(x, n):
  n = n & (2 ** math.floor(math.log2(w)) - 1)
  return ((x << n) | (x >> (w - n))) & (2 ** w - 1)

In [30]:
# Генерация констант:
def gen_const(w):
  f = (math.sqrt(5) + 1) / 2  # золотое сечение
  P = odd((math.e - 2) * 2**w)
  Q = odd((f - 1) * 2**w)
  return P, Q

In [31]:
# Процедура расширения ключа:
def key_expansion(b, key):
  if b == 0:
    c = 1
    L = [b'\x00']
  else:
    if b % w8 != 0:
      key = b'\x00' * (w8 - b % w8) + key   # дополнение ключа до длины кратной w/8
    c = b // w8   # количество блоков по w/8 байт ключа 
    L = [key[w8-1 :: -1]]

  # разбиваем ключ на c блоков:
  for i in range(1, c):
    L.append(key[w8*(i+1)-1 : w8*i-1 : -1])
  L = [int.from_bytes(L[i], byteorder = 'big') for i in range(c)]

  P, Q = gen_const(w)
  S = []
  S.append(P)
  for i in range(2*r+3):
    S.append((S[i] + Q) % 2**w)

  A = B = i = j = 0

  v = 3 * max(c, 2*r+4)
  for k in range(v):
    S[i] = (S[i] + A + B) % 2**w
    A = S[i] = left_shift(S[i], 3)
    L[j] = (L[j] + A + B) % 2**w
    B = L[j] = left_shift(L[j], A + B)
    i = (i+1) % (2*r+4)
    j = (j+1) % c

  return S

In [32]:
# Шифрование блока (4 w-битных регистра):
def enc_round(S, text):
  A = int.from_bytes(text[w8-1::-1], byteorder = 'big')
  B, C, D = map(lambda x: int.from_bytes(x, byteorder = 'big'), [text[w8*(i+1)-1 : w8*i-1 : -1] for i in range(1, 4)])
  B = (B + S[0]) % 2**w
  D = (D + S[1]) % 2**w

  lgw = math.floor(math.log2(w))

  for i in range(r):
    t = left_shift((B * (2*B + 1)) % 2**w, lgw)
    u = left_shift((D * (2*D + 1)) % 2**w, lgw)
    A = (left_shift(A ^ t, u) + S[2*(i+1)]) % 2**w
    C = (left_shift(C ^ u, t) + S[2*(i+1)+1]) % 2**w 
    A, B, C, D = B, C, D, A

  A = (A + S[2*r + 2]) % 2**w
  C = (C + S[2*r + 3]) % 2**w

  return [A, B, C, D]


In [33]:
def encryption(S, intext):
  blocksize = w8 * 4

  if len(intext) % blocksize != 0:
    intext = b'\x00' * (blocksize - len(intext) % blocksize) + intext
  n = len(intext) // blocksize

  ciphertext = []
  for i in range(n):
    ciphertext += enc_round(S, intext[blocksize * i : blocksize * (i+1)])

  CT = str()
  for ct in ciphertext:
    CT += "%02X" % int.from_bytes(ct.to_bytes(w8, byteorder = 'little'), byteorder = 'big')

  return ciphertext, CT

In [34]:
def dec_round(S, text):
  A, B, C, D = text

  C = (C - S[2*r + 3]) % 2**w
  A = (A - S[2*r + 2]) % 2**w

  lgw = int(math.log2(w))

  for i in range(r, 0, -1):
    A, B, C, D = D, A, B, C
    u = left_shift((D * (2*D + 1)) % 2**w, lgw)
    t = left_shift((B * (2*B + 1)) % 2**w, lgw)
    C = right_shift((C - S[2*i + 1]) % 2**w, t) ^ u
    A = right_shift((A - S[2*i]) % 2**w, u) ^ t

  D = (D - S[1]) % 2**w
  B = (B - S[0]) % 2**w

  return [A, B, C, D]

In [35]:
def decryption(S, ciphertext):
  blocksize = w8 * 4
  n = len(ciphertext) // 4
  
  outtext = []
  for i in range(n):
    outtext += dec_round(S, ciphertext[blocksize * i : blocksize * (i+1)])

  OT = str()
  for ot in outtext:
    OT += "%02X" % int.from_bytes(ot.to_bytes(w8, byteorder = 'little'), byteorder = 'big')
  return OT

In [36]:
# RC6-32/20/16
w = 32  # длина машинного слова слова в битах
r = 20  # число раундов
b = 16  # длина ключа в битах
key = 0x102030405060708090A0B0C0D0E0F
key = key.to_bytes(b, byteorder='big')
w8 = w // 8 # длина машинного слова в байтах

M = 0xABCD101112131415161718191A1B1C1D
M = M.to_bytes(math.ceil(M.bit_length()/8), byteorder='big')

In [37]:
print('RC6-32/20/16:')
S = key_expansion(b, key)
ciphertext, CT = encryption(S, M)
print('Ciphertext =', CT)
answer = decryption(S, ciphertext)
print('Message =', answer)

RC6-32/20/16:
Ciphertext = 30623BA4D22984946319C50017D52E8F
Message = ABCD101112131415161718191A1B1C1D


In [38]:
# RC6-8/12/4
w = 8
r = 12
b = 4
key = 0x00010203
key = key.to_bytes(b, byteorder='big')
w8 = w//8

M = 0x00010203
M = M.to_bytes(math.ceil(M.bit_length()/8), byteorder='big')


In [39]:
print('RC6-8/12/4:')
S = key_expansion(b, key)
ciphertext, CT = encryption(S, M)
print('Ciphertext =', CT)
answer = decryption(S, ciphertext)
print('Message =', answer)

RC6-8/12/4:
Ciphertext = AEFC4612
Message = 00010203
